In [ ]:
%%time

import json

import pandas
import requests
from PIL import Image
from adlfs import AzureBlobFileSystem
from tqdm import tqdm

from common.captioning.azure_descriptions import AzureCaption
from common.storage.azure_file_storage import AzureFileStorageAdapter
from common.schemas.pyarrow_schema import tagging_schema

tqdm.pandas(desc="Progress")
file_system: AzureBlobFileSystem = AzureFileStorageAdapter('data').get_file_storage()

from common.functions.functions import Functions

functions: Functions = Functions()

caption: AzureCaption = AzureCaption(file_system)

In [ ]:
%%time

curated_df = pandas.read_parquet('data/parquet/back.parquet', filesystem=file_system, engine='pyarrow')

accepted = curated_df.loc[curated_df["accept"] == True]

dropped = accepted.dropna()
dropped.reset_index(inplace=True, drop=True)

del accepted

accepted = dropped.copy()
del dropped

display(accepted)

In [ ]:
%%time


display("=== Obtaining Caption Files List ===")
current_captions = file_system.ls("data/caption")
display(f"current caption files: {len(current_captions)}")

all_data = []
filtered_data = []
for caption_file in tqdm(current_captions, total=len(current_captions), desc='Reading caption files'):
	image_id = caption_file.split('/')[-1].split('.')[0]
	try:
		file_size = file_system.size(caption_file)
		if file_size == 0:
			display(f'Empty file -- removing {image_id}', clear=True)
			file_system.rm(caption_file)
			continue
		caption_data = json.loads(file_system.read_text(caption_file, encoding='utf-8'))
		caption_data["id"] = image_id
		dense_caption_result = caption_data.get('denseCaptionsResult')
		metadata = caption_data.get('metadata')
		tags_result = caption_data.get('tagsResult')
		smart_crop_result = caption_data.get('smartCropsResult')
		basic_caption = caption_data.get('captionResult')
		_filtered_data = {
			"id": image_id,
			"captions": [basic_caption],
			"dense_captions": dense_caption_result['values'],
			"meta": [metadata],
			"tags": tags_result['values'],
			"smart_crop": smart_crop_result['values']
		}
		all_data.append(caption_data)
		filtered_data.append(_filtered_data)
	except Exception as e:
		display(f" Exception {e} for {image_id}", clear=True)
		continue

In [ ]:
%%time

all_data_from_captions = pandas.DataFrame(data=all_data)
filtered_data_from_captions = pandas.DataFrame(data=filtered_data)

display(all_data_from_captions)
display(filtered_data_from_captions)

all_data_from_captions.to_parquet("data/parquet/all_data_from_captions.parquet", engine='pyarrow',
								  filesystem=file_system)

filtered_data_from_captions.to_parquet("data/parquet/filtered_data_from_captions.parquet", engine='pyarrow',
									   filesystem=file_system)

In [ ]:
single_caption_data = pandas.DataFrame(
	{
		'id': all_data_from_captions['id'],
		'azure_caption': [item['text'] for item in all_data_from_captions['captionResult']],
		'tags': [[foo['name'] for foo in item['values']] for item in all_data_from_captions['tagsResult']]
	})


single_caption_data_indexed = single_caption_data.set_index("id")

accepted_indexed = accepted.set_index("id")

for index, row in accepted_indexed.iterrows():
	accepted_indexed.at[index, 'azure_caption'] = ""
	accepted_indexed.at[index, 'thumbnail_path'] = ""
	accepted_indexed.at[index, 'thumbnail_exists'] = False
	accepted_indexed.at[index, 'thumbnail_curated'] = False
	accepted_indexed.at[index, 'thumbnail_accept'] = False

accepted_indexed.update(single_caption_data_indexed)

accepted_final = accepted_indexed.reset_index()

display("== Updated With Basic Captions ==")
display(accepted_final)

In [ ]:
accepted_final.to_parquet("data/parquet/curation_2.parquet", engine='pyarrow', filesystem=file_system, schema=tagging_schema)
del accepted_final
accepted_final = pandas.read_parquet("data/parquet/curation_2.parquet", engine='pyarrow', filesystem=file_system, schema=tagging_schema)
display(accepted_final)

In [ ]:
# TODO Get dataframe for crops

In [ ]:
import os

def create_thumbnail(_image_id: str, _curated_df: pandas.DataFrame, _crops: pandas.DataFrame, _extant_file_names: list):
	_file_system: AzureBlobFileSystem = AzureFileStorageAdapter('data').get_file_storage()
	out_path = f"data/image/thumbnail/{_image_id}.jpg"
	try:
		if _image_id is None or _image_id in _extant_file_names:
			display(f'Image {_image_id} already exists, skipping', clear=True)
			return out_path

		cropping_information = _crops.loc[_crops['id'] == _image_id]
		if cropping_information is None or len(cropping_information) == 0:
			display(f'No cropping information for {_image_id}, skipping', clear=True)
			return ""

		record = _curated_df.loc[_curated_df['id'] == _image_id]
		record_path = record.to_dict(orient='records')[0]['path']
		image_url = file_system.url(record_path)
		original_image = Image.open(requests.get(image_url, stream=True).raw)
		copied_image = original_image.copy()
		original_image.close()

		cropped = copied_image.crop((cropping_information['smart_crop_boundingBox.x'].values[0],
									 cropping_information['smart_crop_boundingBox.y'].values[0],
									 cropping_information['smart_crop_boundingBox.x'].values[0] +
									 cropping_information['smart_crop_boundingBox.w'].values[0],
									 cropping_information['smart_crop_boundingBox.y'].values[0] +
									 cropping_information['smart_crop_boundingBox.h'].values[0]))
		copied_image.close()

		resized = cropped.resize((512, 512), 1)
		resized.save('temp.jpg')
		resized.close()
		file_system.upload('temp.jpg', out_path, overwrite=True)
		display(f'Thumbnail created for {_image_id}', clear=True)
		return out_path

	except Exception as ex:
		display(f'Error creating thumbnail for {_image_id}: {ex}', clear=True)
		return ""

In [ ]:
records = d.to_dict(orient='records')

for elem in tqdm(records, len(records)):
	create_thumbnail(elem['id'], d, )

In [ ]:
!jupyter notebook stop